# Jupyter Notebook approach for SCORE interaction


### Necessary packages 

In [1]:
from checkscore.methods import readonly, external, payable
from checkscore.methods import Checkscore
from iconsdk.wallet.wallet import KeyWallet

password = input()
deployer_wallet = KeyWallet.load("keystore_ibriz.json",password)
deployer_wallet.get_address()

SCORE_ADDRESS = ""
Checkscore(SCORE_ADDRESS).fill_methods()

ModuleNotFoundError: No module named 'checkscore'

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [16]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
import requests
import json
import pickle
import csv
import os

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

mainnet= {
    "iconservice":"https://ctz.solidwallet.io",
    "nid" :1
    
}

yeouido={
     "iconservice":"https://bicon.net.solidwallet.io",
     "nid" : 3
    
}
euljiro={
     "iconservice":"https://test-ctz.solidwallet.io",
     "nid" : 2
    
}
pagoda={
    "iconservice":"https://zicon.net.solidwallet.io",
     "nid" :80
    
    
}
custom={
    "iconservice":"http://18.237.205.52:9000/",
     "nid":3
    
    
}


### Required service

In [21]:
env = yeouido #mainnet,yeouido,euljiro,pagoda,custom
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]



### A list for SCORE addresses 

In [39]:
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

# Yeouido Testnet
# SCORE_ADDRESS = "cxf42190e91701bec907c7c4514eda6ee0af7f795b"
SAMPLE_TOKEN="cx34b763e96dec767254b5c20edc6ba362ca6cec9e"
oUSDb = "cx6ecc3d356cbf4dbd9c5d273bc391e8b5ca65feb2"
LENDING_POOL="cxbe82d9312bc5b904b4908add8a0ba9a714ae0c1d"
LENDING_POOL_CORE="cx2459bd8744dcb90d469073e1c6a11fb31ad61eef"
LENDING_POOL_DATA_PROVIDER="cx54d0cdc85326c6be067f2120f2d9ff6cd6929696"
PRICE_ORACLE="cxdd8b594ffe5b537069346bab6e4c8bf16ba291ad"
ADDRESS_PROVIDER="cx8cf852997cfdadabd6d0c9bcb107d7db177d4404"
Wallet3_priv_key="eaffd6206b461cd001aaa366a58ad60ffc36e1c6b5b18e315cfe6a2b36e4cae8"

# Mainnet}
# SCORE_ADDRESS = "cx346a2c30ea2712557deb4d7392688750350d1dea"

### Importing necessary wallets

If you want to use your pre-exisiting wallets, execute the cell below:-

In [23]:
# password = input()
private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

In [43]:
deployer_wallet.get_private_key()

'a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a'

### OR creating new wallets 

If you want to use new wallets, execute the cell below:-

### Loading ICX
Go to [ibriz faucet](https://icon-faucet.ibriz.ai/) to load ICX into your wallets.

### Deploying contract

In [31]:
# params = {'_name':"USDbTokenBearing", '_decimals': 18,'_symbol':'oUSDb'}
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('addressProvider'))\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xe783f2fa85b175fe9ecbc54085b06d7f680a85c312d010c333704723bfbacb47',
 'blockHeight': 8883842,
 'blockHash': '0x2eeb45d387125a9ad8ea3864d4d02f0945d99d40975a0cfebde641e0811c869d',
 'txIndex': 2,
 'to': 'cx0000000000000000000000000000000000000000',
 'scoreAddress': 'cx8cf852997cfdadabd6d0c9bcb107d7db177d4404',
 'stepUsed': 0,
 'stepPrice': 0,
 'cumulativeStepUsed': 0,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

### Updating contract

In [42]:
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LENDING_POOL)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('lendingPool'))\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)


get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x765dd8444d68691ba214981a06058abc5d5fcb0918becd7cf0ad4cc30249d025',
 'blockHeight': 8896916,
 'blockHash': '0x2ca2bf695c1a6cab37d2dff323b2c043606c8b31bbbe17d24e7221d41b9759d6',
 'txIndex': 1,
 'to': 'cxbe82d9312bc5b904b4908add8a0ba9a714ae0c1d',
 'scoreAddress': 'cxbe82d9312bc5b904b4908add8a0ba9a714ae0c1d',
 'stepUsed': 0,
 'stepPrice': 0,
 'cumulativeStepUsed': 0,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

# Setting symbol

In [13]:
params ={"_reserveAddress":"cx072d8f95877a53df350e3dc3d8dba2f379037d42","_sym":"USDb" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LENDING_POOL_DATA_PROVIDER) \
    .nid(NID) \
    .nonce(100) \
    .method("setSymbol")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx84490f03cc04b799fe668545340eaeac7cae628c', 'stepLimit': '0x3c618', 'timestamp': '0x5b30a3728b128', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'setSymbol', 'params': {'_reserveAddress': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42', '_sym': 'USDb'}}, 'signature': '1535yLs0HCmecpllTL5zWx9RULWCaPt8cPlOOeaF3TZUBgjVLjTTy3XdVbQOYvRjii8xGePSbmTi+k9eJlgFTQA='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xe7b7ea5eec172f11f77ad1e40a3ced5e27e04aefee433db58292bed46f3a4006',
 'blockHeight': 97841,
 'blockHash': '0x1aeffab680d31e8d320bd432bec4ae04914b5311230d325fa25b0e99e058e708',
 'txIndex': 1,
 'to': 'cx84490f03cc04b799fe668545340eaeac7cae628c',
 'stepUsed': 147320,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 147320,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [15]:
from pprint import pprint
params= {"_reserve":"cx072d8f95877a53df350e3dc3d8dba2f379037d42"}
call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(ADDRESS_PROVIDER)\
    .method("getAllAddresses")\
    .build()
response = icon_service.call(call)
pprint(response)

{'collateral': {'USDb': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42',
                'sICX': ''},
 'oTokens': {'oICX': '', 'oUSDb': 'cx48dd26868c25cfa744aab9f2fccd76ed40c41454'},
 'systemContract': {'LendingPool': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832',
                    'LendingPoolDataProvider': 'cx84490f03cc04b799fe668545340eaeac7cae628c'}}


# Setting  Addressess

In [14]:
params ={"_address":"cx48dd26868c25cfa744aab9f2fccd76ed40c41454"}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(ADDRESS_PROVIDER) \
    .nid(NID) \
    .nonce(100) \
    .method("setoUSDb")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cxad29953f5b8957704e9092632bccc3b1aac62eea', 'stepLimit': '0x3cde8', 'timestamp': '0x5b318213950a2', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'setoUSDb', 'params': {'_address': 'cx48dd26868c25cfa744aab9f2fccd76ed40c41454'}}, 'signature': '8+n+iQJexwiTc9c+cUNlxee1Fx+eh0lX76/hdwCXCsIND79PtKSRp5DvTDZGI+IUPl+b45EFWtHzZWYXYuSBXwE='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x0c2bfd5f0f7819c85dc2946f8484773296ba32abae45dbf409a2c715b822bd4b',
 'blockHeight': 127685,
 'blockHash': '0x0da223d4633b461411726abe56284f723047620ec5d2c119e6e3e33a30798893',
 'txIndex': 1,
 'to': 'cxad29953f5b8957704e9092632bccc3b1aac62eea',
 'stepUsed': 149320,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 149320,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

## Adding new reserve in LendingPoolCore



In [38]:
params ={"_reserve": {"reserveAddress":"cx34b763e96dec767254b5c20edc6ba362ca6cec9e","oTokenAddress":"cx6ecc3d356cbf4dbd9c5d273bc391e8b5ca65feb2","totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"60","liquidationThreshold":"65","liquidationBonus":"10","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LENDING_POOL_CORE) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx2459bd8744dcb90d469073e1c6a11fb31ad61eef', 'stepLimit': '0x186a0', 'timestamp': '0x5b31c29963501', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'addReserveData', 'params': {'_reserve': {'reserveAddress': 'cx34b763e96dec767254b5c20edc6ba362ca6cec9e', 'oTokenAddress': 'cx6ecc3d356cbf4dbd9c5d273bc391e8b5ca65feb2', 'totalBorrows': '0', 'lastUpdateTimestamp': '0', 'liquidityRate': '0', 'borrowRate': '0', 'liquidityCumulativeIndex': '1000000000000000000', 'borrowCumulativeIndex': '1000000000000000000', 'baseLTVasCollateral': '60', 'liquidationThreshold': '65', 'liquidationBonus': '10', 'decimals': '18', 'borrowingEnabled': '1', 'usageAsCollateralEnabled': '1', 'isFreezed': '0', 'isActive': '1'}}}, 'signature': 'SQSHS/DontTvjNR/o95i3AfnvCj9zXWUkZD7nGL8A7xhrX7uChpSeKD1Bu5ZGzCqpDdUbhhR43/whbiJ1qAcrAE='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -3

{'txHash': '0xcd5474aedb42750379a26bae6ae56b4700297c3e4f434e3147f42f5d3214eeb6',
 'blockHeight': 8884297,
 'blockHash': '0xa9f182dd1d591a0acc77686955d437c1eb3c0ec2870ffca79e919e91b1c85096',
 'txIndex': 2,
 'to': 'cx2459bd8744dcb90d469073e1c6a11fb31ad61eef',
 'stepUsed': 0,
 'stepPrice': 0,
 'cumulativeStepUsed': 0,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Setting reserve constants in LendingPoolCore

In [36]:
params ={"_constants": [{"reserve":"cx34b763e96dec767254b5c20edc6ba362ca6cec9e","optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":"0","slopeRate1":f"7{'0'*16}","slopeRate2":f"3{'0'*18}"} ]}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(LENDING_POOL_CORE) \
    .nid(NID) \
    .nonce(100) \
    .method("setReserveConstants")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx2459bd8744dcb90d469073e1c6a11fb31ad61eef', 'stepLimit': '0x186a0', 'timestamp': '0x5b31c22f3dfda', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'setReserveConstants', 'params': {'_constants': [{'reserve': 'cx34b763e96dec767254b5c20edc6ba362ca6cec9e', 'optimalUtilizationRate': '600000000000000000', 'baseBorrowRate': '0', 'slopeRate1': '70000000000000000', 'slopeRate2': '3000000000000000000'}]}}, 'signature': 'CK3+MZViWFpxsAtvEJXRGYRR24e39u9AB84yY1VyGENPURA/AmtPH5Be7UzUe03MKnVikJLCOnORP4fiechl4AE='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending trans

{'txHash': '0x2a35b91aef6fb9cb3878e2383a22d20486df8f5cf37f8589c11655ab1f435f61',
 'blockHeight': 8884242,
 'blockHash': '0x5fe3bee41a5420ef29a3232b6c576134c26a67f1adfd046b04cbca1a26c769da',
 'txIndex': 1,
 'to': 'cx2459bd8744dcb90d469073e1c6a11fb31ad61eef',
 'stepUsed': 0,
 'stepPrice': 0,
 'cumulativeStepUsed': 0,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

# Transferrng USDB

In [ ]:

data = "{\"method\": \"deposit\", \"params\": {\"amount\": 5000000000000000000000}}".encode("utf-8")
params = {"_to": "cxbe82d9312bc5b904b4908add8a0ba9a714ae0c1d", "_value": 5000000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(wallet3.get_address())\
    .to('cx34b763e96dec767254b5c20edc6ba362ca6cec9e')\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet3)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

## Setting interfaces in LendingPool

In [10]:
params ={"_address": "cx162fe5b5d3276e9493de1fd440511f0844ca9a7c" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxbf1a257dcb378b0fd41066fa87abef3f25992832") \
    .nid(NID) \
    .nonce(100) \
    .method("setLendingPoolCoreAddress")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832', 'stepLimit': '0x3db30', 'timestamp': '0x5b30902f19fad', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'setLendingPoolCoreAddress', 'params': {'_address': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c'}}, 'signature': '9FBOBTy4THd08vYX10xdGDs7kCmf3A++5OrFGxMPsIliQJQkOOirZ7EY01llX27DV+lOY8Y21skkp1GublMWUwE='}


{'txHash': '0x5ca1ed7362148f9b117259ef94d858e6402732d2fdf9b26bc7aabde69477fc96',
 'blockHeight': 95258,
 'blockHash': '0xedf8170764a54de4541ceb21875022bedecfdd2433bd757018d36145c020fd49',
 'txIndex': 1,
 'to': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832',
 'stepUsed': 152720,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 152720,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [12]:
params ={"_address": "cx072d8f95877a53df350e3dc3d8dba2f379037d42" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxbf1a257dcb378b0fd41066fa87abef3f25992832") \
    .nid(NID) \
    .nonce(100) \
    .method("setUSDbAddress")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832', 'stepLimit': '0x3d298', 'timestamp': '0x5b309044d16af', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'setUSDbAddress', 'params': {'_address': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42'}}, 'signature': 'QudO9QJkDPtvc5UFbQIgBvjDMc/Vj7RJ5zlKcljU6PU9qKdNNaXZIVLeuAWsCCoNGLaFg88mm+pt3L7QhVeDFgE='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x6d25b73a67de8cc1c15422d1bd91b24d987fd7049fe1c2ff095231082048ef2b',
 'blockHeight': 95270,
 'blockHash': '0xa6c15d3ae47aeac3a6d0159aebb685bb1cadad28aee31bf411631542e1f621db',
 'txIndex': 1,
 'to': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832',
 'stepUsed': 150520,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 150520,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Setting interfaces in LendingPoolDataProvider

In [14]:
params ={"_address": "cx162fe5b5d3276e9493de1fd440511f0844ca9a7c" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx84490f03cc04b799fe668545340eaeac7cae628c") \
    .nid(NID) \
    .nonce(100) \
    .method("setLendingPoolCoreAddress")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx84490f03cc04b799fe668545340eaeac7cae628c', 'stepLimit': '0x3db30', 'timestamp': '0x5b30905a139f8', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'setLendingPoolCoreAddress', 'params': {'_address': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c'}}, 'signature': 'sATOVxx94uGnvFWux4iUTuPReQMw4M1yVjPo1GBLSdxH+m8AQwftr5gJh86bX5sD7T1/Lh2FlIL/YBPj94USdAE='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x78af14823370c3c75204802d846d3f9cf849e8016b741b03e0c953d0b75b14b2',
 'blockHeight': 95281,
 'blockHash': '0xdb46541a96584cfcaba5ac4a106349cf6eedf765561f2f824721b37b25e4ac83',
 'txIndex': 1,
 'to': 'cx84490f03cc04b799fe668545340eaeac7cae628c',
 'stepUsed': 152720,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 152720,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [15]:
params ={"_address": "cx293582520a75648d91a3c454ad9657aa4cbf1f7d" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx84490f03cc04b799fe668545340eaeac7cae628c") \
    .nid(NID) \
    .nonce(100) \
    .method("setOracleAddress")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)


{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx84490f03cc04b799fe668545340eaeac7cae628c', 'stepLimit': '0x3d428', 'timestamp': '0x5b30906951406', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'setOracleAddress', 'params': {'_address': 'cx293582520a75648d91a3c454ad9657aa4cbf1f7d'}}, 'signature': 'v4cU6qZb3shonfTy/1wZHgeAyWKfqj2Ynv91VJMOwzoIL6zOdtlAnkotsc4zIlBWvz7CbMsweBL8RN+A/L0sSgA='}


## Setting Price in Oracle

In [18]:
params ={"_base": "USDb", "_quote":"USD","_rate": 5 * 10**17 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx293582520a75648d91a3c454ad9657aa4cbf1f7d") \
    .nid(NID) \
    .nonce(100) \
    .method("set_reference_data")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx293582520a75648d91a3c454ad9657aa4cbf1f7d', 'stepLimit': '0x3c708', 'timestamp': '0x5b3090ef50dfe', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'set_reference_data', 'params': {'_base': 'USDb', '_quote': 'USD', '_rate': '0x6f05b59d3b20000'}}, 'signature': 'tyWQsOlMOTprefx9NKpxGzxkh2jZYMyew7ZuGuyfS29lfH6nCHRZtj+dkVfKvNTRzXcs4CG4DW3cPMK3ptEPbwA='}


### ICX Transfer

In [ ]:
transaction = TransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("hx233633ad3d85fe2aed5ac71863ac5d7a2f31d16b")\
    .nid(3) \
    .nonce(100) \
    .value(10000*10**18)\
    .build()

estimate_step = icon_service.estimate_step(transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

In [ ]:
balance = icon_service.get_balance("hxedcdfa97532eb690cf65c0f27ca1b5d4374b8122")
print(balance)

In [37]:
# params = {'_address': 'hxc9319d218ffff1672c9f1c15365c6f6c0d161c7b' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxad29953f5b8957704e9092632bccc3b1aac62eea")\
    .method("getAllAddress")\
    .build()
response = icon_service.call(_call)
print(response)

{'collateral': {'USDb': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42', 'sICX': ''}, 'oTokens': {'oUSDb': 'cx48dd26868c25cfa744aab9f2fccd76ed40c41454', 'oICX': ''}, 'systemContract': {'LendingPool': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832', 'LendingPoolDataProvider': 'cx84490f03cc04b799fe668545340eaeac7cae628c'}}
